In [1]:
@file:DependsOn("ro.jf.funds:user-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:reporting-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
@file:DependsOn("com.charleskorn.kaml:kaml-jvm:0.67.0")
@file:DependsOn("ro.jf.funds:funds-notebook-client:1.0.0")
@file:DependsOn("io.ktor:ktor-client-core:3.1.3")
@file:DependsOn("io.ktor:ktor-client-cio:3.1.3")
@file:DependsOn("io.ktor:ktor-client-content-negotiation:3.1.3")
@file:DependsOn("io.ktor:ktor-serialization-kotlinx-json:3.1.3")
%use dataframe
%use kandy

import ro.jf.funds.client.notebook.*
import ro.jf.funds.reporting.api.model.*

val client = FundsClient()
val user = client.ensureUserExists("Johann-19.1")

val granularInterval = GranularDateInterval(
    interval = DateInterval(
        LocalDate.parse("2019-01-01"),
        LocalDate.parse("2020-07-31")
    ),
    granularity = TimeGranularity.MONTHLY
)

user

UserTO(id=aa22e729-8911-4a55-9e67-3adcede9d032, username=Johann-19.1)

In [2]:
import ro.jf.funds.user.sdk.UserSdk
import kotlinx.coroutines.runBlocking

val REPORT_DATA_CONFIGURATION_YAML_FILE = "../../data/provision/expenses-report-data-configuration.yaml"

In [3]:
import ro.jf.funds.fund.sdk.FundSdk
import ro.jf.funds.fund.api.model.FundName

val fundSdk = FundSdk()
val expenseFund = runBlocking { fundSdk.getFundByName(user.id, FundName("Expenses")) }
expenseFund

FundTO(id=0c4e4701-ad45-405d-b8b2-045dd89aacaa, name=Expenses)

In [4]:
import ro.jf.funds.reporting.api.model.*
import kotlinx.serialization.json.Json
import com.charleskorn.kaml.Yaml

fun getReportDataConfiguration(): ReportDataConfigurationTO {
    val dataConfigurationRawJson: String = File(REPORT_DATA_CONFIGURATION_YAML_FILE).readText()
    val dataConfiguration = Yaml.default.decodeFromString(ReportDataConfigurationTO.serializer(), dataConfigurationRawJson)
    return dataConfiguration
}
val dataConfiguration = getReportDataConfiguration()
dataConfiguration

ReportDataConfigurationTO(currency=Currency(value=RON), filter=RecordFilterTO(labels=[basic, home, shopping_services, transport, fun, gifts, development]), groups=[ReportGroupTO(name=basic, filter=RecordFilterTO(labels=[basic])), ReportGroupTO(name=home, filter=RecordFilterTO(labels=[home])), ReportGroupTO(name=shopping_services, filter=RecordFilterTO(labels=[shopping_services])), ReportGroupTO(name=transport, filter=RecordFilterTO(labels=[transport])), ReportGroupTO(name=fun, filter=RecordFilterTO(labels=[fun])), ReportGroupTO(name=gifts, filter=RecordFilterTO(labels=[gifts])), ReportGroupTO(name=development, filter=RecordFilterTO(labels=[development])), ReportGroupTO(name=investment, filter=RecordFilterTO(labels=[investment]))], features=ReportDataFeaturesConfigurationTO(net=NetReportFeatureTO(enabled=true, applyFilter=true), valueReport=GenericReportFeatureTO(enabled=true), groupedNet=GenericReportFeatureTO(enabled=true), groupedBudget=GroupedBudgetReportFeatureTO(enabled=true, dist

In [5]:
import ro.jf.funds.reporting.sdk.ReportingSdk
import kotlinx.coroutines.delay
import ro.jf.funds.commons.model.labelsOf
import ro.jf.funds.commons.model.Currency
import ro.jf.funds.reporting.api.model.ReportViewTaskStatus
import java.time.Instant

val reportingSdk = ReportingSdk()
val reportView = runBlocking {
    val reportViewName = "Expense report"
    val existingReportView = reportingSdk.listReportViews(user.id).items.firstOrNull { it.name == reportViewName }
    if (existingReportView != null) {
        return@runBlocking existingReportView
    }
    val request = CreateReportViewTO(reportViewName, expenseFund.id, dataConfiguration)
    var task: ReportViewTaskTO = reportingSdk.createReportView(user.id, request)
    val timeout = Instant.now().plusSeconds(120)
    while (task.status == ReportViewTaskStatus.IN_PROGRESS && Instant.now().isBefore(timeout)) {
        delay(2000)
        task = reportingSdk.getReportViewTask(user.id, task.taskId)
    }
    if (task.status == ReportViewTaskStatus.COMPLETED) {
        task.report ?: error("No report found on completed report task")
    } else if (task.status == ReportViewTaskStatus.FAILED) {
        throw IllegalStateException("Report view creation failed on task $task")
    } else {
        throw IllegalStateException("Report view creation timed out on task $task")
    }
}
reportView

ReportViewTO(id=1ddc6f5b-2b19-4107-87a5-2c0e921ebc21, name=Expense report, fundId=0c4e4701-ad45-405d-b8b2-045dd89aacaa, dataConfiguration=ReportDataConfigurationTO(currency=Currency(value=RON), filter=RecordFilterTO(labels=[basic, home, shopping_services, transport, fun, gifts, development]), groups=null, features=ReportDataFeaturesConfigurationTO(net=NetReportFeatureTO(enabled=true, applyFilter=true), valueReport=GenericReportFeatureTO(enabled=true), groupedNet=GenericReportFeatureTO(enabled=false), groupedBudget=GroupedBudgetReportFeatureTO(enabled=false, distributions=[]), forecast=ForecastReportFeatureTO(enabled=false, inputBuckets=0, outputBuckets=0))))

In [6]:
import ro.jf.funds.reporting.api.model.*

val report = runBlocking {
    reportingSdk.getReportViewData(user.id, reportView.id, granularInterval)
}
report

ReportDataTO(viewId=1ddc6f5b-2b19-4107-87a5-2c0e921ebc21, granularInterval=GranularDateInterval(interval=DateInterval(from=2019-01-01, to=2020-07-31), granularity=MONTHLY), data=[ReportDataItemTO(timeBucket=DateInterval(from=2019-01-01, to=2019-01-31), bucketType=REAL, net=-4468.51, value=ValueReportTO(start=0.0, end=2481.52, min=0.0, max=0.0), groups=[ReportDataGroupItemTO(group=basic, net=-2305.04, allocated=2085.009, left=-220.031), ReportDataGroupItemTO(group=home, net=-500.0, allocated=2085.009, left=1585.009), ReportDataGroupItemTO(group=shopping_services, net=-385.57, allocated=347.5015, left=-38.0685), ReportDataGroupItemTO(group=transport, net=-204.24, allocated=347.5015, left=143.2615), ReportDataGroupItemTO(group=fun, net=-1031.16, allocated=1042.5045, left=11.3445), ReportDataGroupItemTO(group=gifts, net=0.0, allocated=834.0036, left=834.0036000000001), ReportDataGroupItemTO(group=development, net=-42.5, allocated=208.5009, left=166.0009), ReportDataGroupItemTO(group=invest

In [7]:
import org.jetbrains.kotlinx.kandy.ir.Plot

fun plotGroupData(plotTitle: String, groupItemFilter: (ReportDataGroupItemTO) -> Boolean): Plot =
    dataFrameOf(
        "month" to report.data.map<ReportDataItemTO, LocalDate> { it.timeBucket.from },
        "spent" to report.data.map { bucket -> bucket.groups!!.filter(groupItemFilter).map { it.net!!.negate() }.reduce { acc, value -> acc + value } },
        "allocated" to report.data.map { bucket -> bucket.groups!!.filter(groupItemFilter).map { it.allocated!! }.reduce { acc, value -> acc + value } },
        "left" to report.data.map { bucket -> bucket.groups!!.filter(groupItemFilter).map { it.left!! }.reduce { acc, value -> acc + value } },
    )
        .plot {
            x("month")
            line {
                y.constant(0)
            }
            line {
                y("spent")
                color = Color.RED
            }
            line {
                y("allocated")
                color = Color.GREEN
            }
            area {
                y("left")
                borderLine {
                    color = Color.YELLOW
                }
            }
            layout {
                title = plotTitle
                size = 1200 to 600
            }
        }

fun plotGroup(title: String, group: String): Plot =
    plotGroupData(title) { it.group == group }

fun plotGroupsTotal(): Plot =
    plotGroupData("Total expenses") { true }



In [8]:
plotGroupsTotal()

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="DygWiY"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Total expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12],
"left":[2481.52,5185.08,11301.259999999998,11526.016022686399,10643.310981436,11868.237464847596,8637.902755956795,-6550.839311906806,-872.2869408013057,4051.7634872546964,6037.1198616281945,12394.376169342593,18628.670795337093,24174.782837978095,22045.408532850495,24075.588180919895,17645.084581462583,6743.352806605984,-10496.233404802118,9823.065632989115,11187.557711730444,12192.544766108089,12870.943206012538,13440.428484711234,13527.599510991955],
"spent":[4468.51,4301.32,9171.45,6967.76,9043.57,7024.09,11232.58,25034.218092,4537.4484992,6776.5,9787.68,5529.22,4702.18,5442.03,35747.94,17363.44,18069.43,20971.82,27334.72,15108.052215933334,14280.871726261112,15092.823661849543,15785.850633670336,16285.698186476193,17182.071368682547],
"allocated":[6950.03,7004.88,15287.63,7192.5160226864,8165.19,8255.89,7999.83,9846.81,10219.0435174615,11703.94,11782.32,11887.53,10934.14,10999.55,33626.01,19395.52,11641.82,10066.59,10094.98,13516.521126455125,13822.330386993051,14122.604292787348,14324.159650519627,14535.979621396264,14756.683756512619]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"spent"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"allocated"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"left"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"month"
}]
},
"spec_id":"2"
};
 var containerDiv = document.getElementById("DygWiY");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M50.22922167989964 389.8149558828522 L50.22922167989964 389.8149558828522 L92.83129752332025 389.8149558828522 L131.3105918335059 389.8149558828522 L173.91266767692287 389.8149558828522 L215.14048300926515 389.8149558828522 L257.7425588526821 389.8149558828522 L298.9703741850244 389.8149558828522 L341.572450028445 389.8149558828522 L384.

In [9]:
plotGroup("Basic Expenses", "basic")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="treFQQ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Basic Expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12],
"left":[-220.031,-627.8770000000001,610.492,336.18638493090424,513.6170782982102,442.6164755148786,-1101.755601134993,-940.1643541784716,223.9845507993176,911.0944044881384,860.1852376493127,1313.6531206792206,1670.3263862079468,2033.6424165548842,7442.427933482604,10566.518760679792,11057.505242124238,10019.429217975094,9098.263520613955,4521.405536423003,4976.536360639792,5372.582344793165,5744.373006485251,6151.388653888246,6554.533281655664],
"spent":[2305.04,2509.31,3347.92,2293.91,2108.18,2382.69,3709.61,2493.78,1587.87,2253.23,2992.86,2517.62,2158.1,2165.51,2324.86,948.25,1951.84,3154.12,3041.34,2299.115,2282.892916666667,2340.811493055556,2348.109950810185,2294.380780044367,2275.777511714731],
"allocated":[2085.009,2101.464,4586.289,2013.904486352192,2286.2532,2311.6492,2159.9541,2658.6387,2759.141749714605,2925.985,2945.58,2971.8825,2514.8522,2529.8965,7733.9823,4073.0592,2444.7822,2113.9839,2119.9458,3149.310837476217,3190.200182265902,3226.121718311843,3251.133111504497,3276.595870796538,3301.98865169625]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"spent"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"allocated"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"left"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"month"
}]
},
"spec_id":"5"
};
 var containerDiv = document.getElementById("treFQQ");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M50.59197647333349 454.4017970753655 L50.59197647333349 454.4017970753655 L93.50172396097332 454.4017970753655 L132.25891524013423 454.4017970753655 L175.1686627277777 454.4017970753655 L216.69422481259244 454.4017970753655 L259.6039723002323 454.4017970753655 L301.129534385047 454.4017970753655 L344.0392818726905 454.4017970753655 L386.9490293603303 

In [10]:
plotGroup("Home Expenses", "home")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="KYegSi"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Home Expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12],
"left":[1585.009,3077.313,5899.701999999999,7793.214198358126,9996.035600499506,12161.334148208018,14283.93857538898,-1392.8678137600643,1404.3426549224882,5012.06543760165,8610.110615592816,12242.625406004363,15778.40745235079,16990.725102854718,-608.1874565426414,-9373.823064460152,-17813.229937952136,-31061.79415899321,-50715.290152832684,-4243.909659601172,-4481.496480087931,-4971.983074672132,-5803.987117361614,-7005.16192844115,-8609.144206311608],
"spent":[500.0,609.16,1763.9,400.0,400.0,467.23,420.0,18881.0,423.24,456.78,400.0,400.0,400.0,2736.62,29698.21,16136.26,12866.69,17070.89,23488.88,10246.5475,9527.009791666667,10285.65727430556,11104.73038049769,11996.79124553916,12963.19051600076],
"allocated":[2085.009,2101.464,4586.289,2301.605127259648,2612.8608,2641.8848,2559.9456,3150.9792,3270.09392558768,3979.3396,4005.9888,4041.7602,3936.2904,3959.838,12105.3636,7370.2976,4423.8916,3825.3042,3836.0924,4825.436627132307,4964.974746059999,5106.214814432692,5200.121082302083,5299.632105827257,5404.454764646195]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"spent"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"allocated"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"left"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"month"
}]
},
"spec_id":"8"
};
 var containerDiv = document.getElementById("KYegSi");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M50.22922167989964 198.60458816664874 L50.22922167989964 198.60458816664874 L92.83129752332025 198.60458816664874 L131.3105918335059 198.60458816664874 L173.91266767692287 198.60458816664874 L215.14048300926515 198.60458816664874 L257.7425588526821 198.60458816664874 L298.9703741850244 198.60458816664874 L341.572450028445 198.60458816664874 L384.174525871862 198.6045

In [11]:
plotGroup("Transport Expenses", "transport")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="GMG6pe"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Transport Expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12],
"left":[143.2615,285.6555,-164.08299999999994,59.11724529891865,75.96404148893195,305.0175778505213,735.1525895632738,149.60745000877293,712.8826972742302,1799.9844478449932,583.0794799257205,-131.69723504942107,572.6918795457889,1731.4381688764774,5535.741155631223,8250.567303618009,8659.989644633737,9868.06955714532,10693.09180788276,4035.453863111467,4359.274397536692,4663.140372558564,4901.736699618028,5261.624801259054,5711.06830428476],
"spent":[204.24,207.85,1214.12,281.75,554.64,348.18,288.04,1474.32,331.47,238.42,2502.33,2020.41,608.51,161.2,230.49,0.0,1218.9,202.81,588.5,798.1133333333333,741.7627777777778,775.9538425925927,820.7483294753087,680.616523598251,568.9670672314386],
"allocated":[347.5015,350.244,764.3815,503.476121588048,571.5633,577.9123,719.9847,886.2129,919.713916571535,1287.4334,1296.0552,1307.6283,1312.0968,1319.946,4035.1212,2715.3728,1629.8548,1409.3226,1413.2972,1627.671259714295,1689.459456357152,1753.604918005954,1792.45254450645,1833.818989881988,1877.66821403882]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"spent"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"allocated"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"left"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"month"
}]
},
"spec_id":"11"
};
 var containerDiv = document.getElementById("GMG6pe");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M50.59197647333349 490.1601801437388 L50.59197647333349 490.1601801437388 L93.50172396097332 490.1601801437388 L132.25891524013423 490.1601801437388 L175.1686627277777 490.1601801437388 L216.69422481259244 490.1601801437388 L259.6039723002323 490.1601801437388 L301.129534385047 490.1601801437388 L344.0392818726905 490.1601801437388 L386.9490293603303 490.1601801437388 L

In [12]:
plotGroup("Shopping & Services Expenses", "shopping_services")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="vy3e5u"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Shopping & Services Expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12],
"left":[-38.0685,312.1755,466.787,-580.2546108317958,-1025.8700502201682,-3019.483338380341,-2840.944907918082,-2061.9149863759735,-1282.3864502262843,-739.8963763745863,208.44788878665742,1160.4575889025107,1666.6513838633323,2325.661576073283,3082.7758484187734,4052.264510730198,4633.524159385392,5068.697066188167,5573.561662658785,1973.986989335854,2310.312153978507,2609.703704328906,2888.83704438753,3112.20280735427,3274.84824222525],
"spent":[385.57,0.0,609.77,1481.1,936.99,2494.99,463.18,0.0,82.99,342.57,0.0,0.0,149.99,0.0,1259.78,0.0,0.0,69.0,0.0,158.6941666666667,171.9186805555556,179.3294039351852,165.726020929784,179.5365226739326,194.497899563427],
"allocated":[347.5015,350.244,764.3815,431.550961361184,489.9114,495.3534,639.9864,787.7448,817.52348139692,936.3152,942.5856,951.0024,656.0484,659.973,2017.5606,969.776,582.091,503.3295,504.749,860.72491511641,866.8065913761108,870.9135172077101,865.4633769750192,859.0365250562708,851.3727021442934]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"spent"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"allocated"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"left"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"month"
}]
},
"spec_id":"14"
};
 var containerDiv = document.getElementById("vy3e5u");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M50.59197647333349 330.8885741205483 L50.59197647333349 330.8885741205483 L93.50172396097332 330.8885741205483 L132.25891524013423 330.8885741205483 L175.1686627277777 330.8885741205483 L216.69422481259244 330.8885741205483 L259.6039723002323 330.8885741205483 L301.129534385047 330.8885741205483 L344.0392818726905 330.8885741205483 L386.9490293603303 330.8885741205483 L428.47459144514505 330.8885741205483 L47

In [13]:
plotGroup("Fun Expenses", "fun")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="9sNZPY"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Fun Expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12],
"left":[11.3445,492.0765,1547.3310000000006,375.57174185766087,-795.9502727743126,274.50797957134023,363.8567080501632,-569.6057038035792,-1191.7222071631459,-590.8304533064754,475.133641617575,1445.598265617614,1631.6696446109124,2400.7395485192637,3466.5071321847495,4435.963830409741,5017.156165577087,5521.398148646272,6026.273069396817,2339.023423525569,2581.409184136332,2895.836800077955,3186.392404526657,3412.330634769081,3576.224998865036],
"spent":[1031.16,570.0,1237.89,2109.0,2235.76,0.0,790.09,1980.318092,1753.8784992,405.5,0.0,100.0,579.01,0.0,1287.38,0.0,0.0,0.0,0.0,508.8405492666667,386.2174207055556,272.2456641643519,261.1411361780478,282.9028975262185,298.1448056534034],
"allocated":[1042.5045,1050.732,2293.1445,935.027082949232,1061.4747,1073.2657,879.9813,1083.1491,1124.094786920765,1053.3546,1060.4088,1069.8777,765.3898,769.9685,2353.8207,969.776,582.091,503.3295,504.749,986.667457243397,978.6273203470136,966.5050314658675,959.2675674213565,950.8391313731362,940.919250654231]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"spent"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"allocated"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"left"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"month"
}]
},
"spec_id":"17"
};
 var containerDiv = document.getElementById("9sNZPY");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M50.59197647333349 419.1187717273425 L50.59197647333349 419.1187717273425 L93.50172396097332 419.1187717273425 L132.25891524013423 419.1187717273425 L175.1686627277777 419.1187717273425 L216.69422481259244 419.1187717273425 L259.6039723002323 419.1187717273425 L301.129534385047 419.1187717273425 L344.0392818726905 419.1187717273425 L386.9490293603303 419.1187717273425 L428.47459144514505 419.118

In [14]:
plotGroup("Gifts Expenses", "gifts")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="ZSN8od"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Gifts Expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12],
"left":[834.0036000000001,1282.5892,2969.2547999999992,3426.3284712782524,2476.6123172029324,2137.503656588581,-2528.8414242491667,-1658.3912330579328,-855.1636654301379,266.30007841051787,-2457.415255477468,-1754.7351591069094,-1317.463640073329,-436.4541199169602,3120.8036526055084,5055.709935184996,4330.573095720185,4963.681095549037,5858.242097030291,1259.64057345315,1502.809890662407,1699.307687003452,1818.72498771953,2175.070007985946,2502.553771910351],
"spent":[0.0,392.0,147.85,402.0,1930.0,1330.0,5561.66,204.8,351.0,20.0,3892.49,491.19,766.57,330.2,141.92,198.28,2005.0,475.0,216.0,757.7041666666667,803.7795138888889,841.5111400462963,909.970401716821,661.4271018598894,675.6135270148802],
"allocated":[834.0036,840.5856,1834.5156,863.101922722368,979.8228,990.7068,879.9813,1083.1491,1124.094786920765,1170.394,1178.232,1188.753,1202.7554,1209.9505,3698.8611,2133.5072,1280.6002,1107.3249,1110.4478,1457.33916557673,1488.521671041458,1518.890578051516,1547.931959555809,1578.740289518793,1611.239230312026]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"spent"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"allocated"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"left"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"month"
}]
},
"spec_id":"20"
};
 var containerDiv = document.getElementById("ZSN8od");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M50.59197647333349 354.50915251073786 L50.59197647333349 354.50915251073786 L93.50172396097332 354.50915251073786 L132.25891524013423 354.50915251073786 L175.1686627277777 354.50915251073786 L216.69422481259244 354.50915251073786 L259.6039723002323 354.50915251073786 L301.129534385047 354.50915251073786 L344.0392818726905 354.50915251073786 L386.9490293603303 354.5091525107

In [15]:
plotGroup("Development Expenses", "development")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="GxB0Ii"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Development Expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12],
"left":[166.0009,363.1473,-28.223800000000015,115.85259179433335,-597.0977330591003,-433.2590345054012,-273.50318374338053,-77.50267073955791,115.77547902222638,-2606.9540514095415,-2242.4217464664193,-1881.5258177047845,-1373.6123111683469,-870.9698549835709,5.340267070277169,1088.386904757309,1759.5662119740812,2363.871880095304,2969.624590447958,-62.53509325875548,-61.28779513535528,-76.04306798182041,134.8661806371563,332.9735078957876,517.515118362502],
"spent":[42.5,13.0,850.0,0.0,878.0,1.0,0.0,0.0,7.0,3060.0,0.0,0.0,40.0,48.5,805.3,80.65,27.0,0.0,0.0,339.0375,367.290625,397.31484375,175.4244140625,190.043115234375,205.8800415039062],
"allocated":[208.5009,210.1464,458.6289,143.850320453728,163.3038,165.1178,159.9966,196.9362,204.38087034923,351.1182,353.4696,356.6259,546.707,549.9775,1681.3005,1163.7312,698.5092,603.9954,605.6988,609.3708641957692,643.7404195454166,680.3537153117655,707.7900082544126,737.3167089422803,769.0409430208038]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"spent"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"allocated"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"left"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"month"
}]
},
"spec_id":"23"
};
 var containerDiv = document.getElementById("GxB0Ii");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M50.59197647333349 279.43248416203403 L50.59197647333349 279.43248416203403 L93.50172396097332 279.43248416203403 L132.25891524013423 279.43248416203403 L175.1686627277777 279.43248416203403 L216.69422481259244 279.43248416203403 L259.6039723002323 279.43248416203403 L301.129534385047 279.43248416203403 L344.0392818726905 279.43248416203403 L386.9490293603303 279.43248416203403 L428.47459144514505 279.43248416203403 L471

In [16]:
plotGroup("Investment Expenses", "investment")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="7vBWRB"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Investment Expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12],
"left":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
"spent":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
"allocated":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"spent"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"allocated"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"y":"left"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"month"
}]
},
"spec_id":"26"
};
 var containerDiv = document.getElementById("7vBWRB");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M51.31748606019028 260.5 L51.31748606019028 260.5 L94.84257683627948 260.5 L134.1555620533909 260.5 L177.68065282947646 260.5 L219.80170841923973 260.5 L263.3267991953253 260.5 L305.44785478508857 260.5 L348.97294556117413 260.5 L392.49803633726333 260.5 L434.61909192702296 260.5 L478.14418270311216 260.5 L520.2652382928754 260.5 L563.790329068961 260.5 L607.3154198450466 260.5 L648.0324402484839 260.5 L691.5575310245731 260.5 L733.6785866143327 260.5 L777.203677390422 260.5 L819.3247329801816 260.5 L862.8498237562708 260.5 L906.3749145323563 260.5 L948.4959701221196 260.5 L992.0210608982052 260.5 L1034.1421164879685 260.5 L1077.667207264054 260.5 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(71,71,71)" stroke-opacity="1.0">
 
 
 
 
 
 <path d="M51.31748606019028 260.5 L51.31748606019028 260.5 L94.84257683627948 260.5 L134.1555620533909 260.5 L177.68065282947646 260.5 L219.80170841923973 260.5 L263.3267991953253 260.5 L305.44785478508857 260.5 L348.97294556117413 260.5 L392.49803633726333 260.5 L434.61909192702296 260.5 L4